# Get Map of Moscow Malls

In [15]:
import pandas as pd
import numpy as np
import folium
from lxml import html
import requests
import json

#### Parse info from malls.ru

In [11]:
# create urls
req = ['http://www.malls.ru/rus/malls/?action=city&mcountry=1&city=20&space=&open=1&PAGEN_1=' + str(i) for i in range(1,23)]

# parse info for each mall
names = []
address = []
areas = []

for r in req:
    t1 = requests.get(r).text
    tree = html.fromstring(t1)
    names = names + tree.xpath('//span[@class="h3"]/text()')[:15]
    areas = areas + tree.xpath('//span[@class="red nowrap"]/text()')[:15]
    address = address + tree.xpath('//span[@class="green"]/text()')[:15]

#### Get geolocations

In [16]:
# get urls for getting coordinates from yandex geocoder

address = list(map(lambda x: 'Москва, ' + x, address))
to_yandex = ['https://geocode-maps.yandex.ru/1.x/?format=json&geocode=' + a for a in address]
data = []
# get coordinates
coords = []
for ty in to_yandex:
    t1 = requests.get(ty).text
    j = json.loads(t1)
    data.append(j)
    coords.append(j['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'])

#### Combine into DataFrame and proceed

In [19]:
data = pd.DataFrame([names, address, coords, areas], index = ['names', 'adress', 'coords', 'areas']).transpose()

num_areas = data['areas'].str.replace('м2', '').astype(float, raise_on_error = False)
num_areas = num_areas.str.replace('\D', '').astype(float)
data['areas'] = num_areas

num_c = [[float(c) for c in reversed(cc)] for cc in data['coords'].str.split(' ').values]
data['coords'] = num_c

#### Draw map

In [33]:
map_osm = folium.Map(location=[55.75, 37.7])
[folium.CircleMarker(d[2], radius = (d[3]/1000) **0.5 , color='#3186cc',
                    fill_color='#3186cc', popup = d[0] + ': ' + str(d[3]/1000) + 'k m2').add_to(map_osm) for d in data.values]
map_osm.save('Moscow_malls.html')
map_osm;

#### Страница:



In [36]:
HTML('<iframe src=malls.html width=1000 height=1000></iframe>')